# Additional functions I used
This file contains the functions used for calculating both the word error rate and character error rate. <br/>
It also contains the code I used to create the transcriptions belonging to the audio files. 

# Functions for calculating the WER and CER
It uses the calculate_wer function from espnet:
https://github.com/espnet/espnet/blob/master/espnet/nets/e2e_asr_common.py

In [57]:
def calculate_wer(seqs_hat, seqs_true):
    #This function originates from: https://github.com/espnet/espnet/blob/master/espnet/nets/e2e_asr_common.py
    """Calculate sentence-level WER score.
    :param list seqs_hat: prediction
    :param list seqs_true: reference
    :return: average sentence-level WER score
    :rtype float
    """
    import editdistance

    word_eds, word_ref_lens = [], []
    for i, seq_hat_text in enumerate(seqs_hat):
        seq_true_text = seqs_true[i]
        hyp_words = [item.lower() for item in seq_hat_text.split()]
        ref_words = [item.lower() for item in seq_true_text.split()]
        word_eds.append(editdistance.eval(hyp_words, ref_words))
        word_ref_lens.append(len(ref_words))
    return round((float(sum(word_eds)) / sum(word_ref_lens)) * 100, 2)

def open_folder_and_return_lines(path):
    with open(path) as f:
        lines = f.readlines()
    return lines

def calculate_wer_character_level(seqs_hat, seqs_true):
    #same function but changed to make it look at character level
    import editdistance
    
    word_eds, word_ref_lens = [], []
    for i, seq_hat_text in enumerate(seqs_hat):
        seq_true_text = seqs_true[i]
        hyp_words = [item.lower() for item in seq_hat_text.split()]
        ref_words = [item.lower() for item in seq_true_text.split()]
        
        #instead of having a list of words, it has all the words in a single sentence
        hyp_words = " ".join(hyp_words)
        ref_words = " ".join(ref_words)
        word_eds.append(editdistance.eval(hyp_words, ref_words))
        word_ref_lens.append(len(ref_words))
    return round((float(sum(word_eds)) / sum(word_ref_lens)) * 100, 2)

## WER for speaker M07

In [58]:
folder = "predictions/"
paths = ['M07_0.75x', 'M07_original', 'M07_1.25x', 'M07_1.5x', 'M07_1.75x', 'M07_2x']
adds = ["_predictions_original.txt", "_predictions_transformed.txt", "_transcription.txt"]

lowest = 1000
path_lowest = "path"
for path in paths:
    transcriptions = open_folder_and_return_lines(folder + paths[0] + adds[2])
    original = open_folder_and_return_lines(folder + path + adds[0])
    converted = open_folder_and_return_lines(folder + path + adds[1])
    
    print(path)
    print("original")
    print(calculate_wer(original, transcriptions))
    print("\n")
    
    print(path)
    print("converted")
    print(calculate_wer(converted, transcriptions))
    print("\n")
    
    if calculate_wer(original, transcriptions) < lowest:
        path_lowest = path + "original"
        lowest =  calculate_wer(original, transcriptions)
    if calculate_wer(converted, transcriptions) < lowest:
        path_lowest = path + "converted"
        lowest = calculate_wer(converted, transcriptions)

print("lowest value")
print(path_lowest, lowest)

print("control group:")
transcriptions = open_folder_and_return_lines("transcriptions_control.txt")
predictions = open_folder_and_return_lines("predictions_control.txt")
print(calculate_wer(predictions, transcriptions))

M07_0.75x
original
237.37


M07_0.75x
converted
268.4


M07_original
original
232.49


M07_original
converted
267.11


M07_1.25x
original
230.92


M07_1.25x
converted
252.04


M07_1.5x
original
224.59


M07_1.5x
converted
228.74


M07_1.75x
original
213.05


M07_1.75x
converted
208.96


M07_2x
original
202.46


M07_2x
converted
195.52


lowest value
M07_2xconverted 195.52
control group:
69.69


## CER for speaker M07

In [59]:
folder = "predictions/"
paths = ['M07_0.75x', 'M07_original', 'M07_1.25x', 'M07_1.5x', 'M07_1.75x', 'M07_2x']
adds = ["_predictions_original.txt", "_predictions_transformed.txt", "_transcription.txt"]

lowest = 1000
path_lowest = "path"
for path in paths:
    transcriptions = open_folder_and_return_lines(folder + paths[0] + adds[2])
    original = open_folder_and_return_lines(folder + path + adds[0])
    converted = open_folder_and_return_lines(folder + path + adds[1])
    
    print(path)
    print("original")
    print(calculate_wer_character_level(original, transcriptions))
    print("\n")
    
    print(path)
    print("converted")
    print(calculate_wer_character_level(converted, transcriptions))
    print("\n")
    
    if calculate_wer_character_level(original, transcriptions) < lowest:
        path_lowest = path + "original"
        lowest =  calculate_wer_character_level(original, transcriptions)
    if calculate_wer_character_level(converted, transcriptions) < lowest:
        path_lowest = path + "converted"
        lowest = calculate_wer_character_level(converted, transcriptions)

print("lowest value")
print(path_lowest, lowest)

print("control group:")
transcriptions = open_folder_and_return_lines("transcriptions_control.txt")
predictions = open_folder_and_return_lines("predictions_control.txt")
print(calculate_wer_character_level(predictions, transcriptions))

M07_0.75x
original
169.95


M07_0.75x
converted
195.05


M07_original
original
162.03


M07_original
converted
189.44


M07_1.25x
original
160.11


M07_1.25x
converted
174.74


M07_1.5x
original
152.16


M07_1.5x
converted
159.15


M07_1.75x
original
145.44


M07_1.75x
converted
147.03


M07_2x
original
139.2


M07_2x
converted
136.72


lowest value
M07_2xconverted 136.72
control group:
30.53


## Creating the transcriptions for experiment M5

In [63]:
import pandas as pd
df = pd.read_excel('speaker_wordlist.xls', sheet_name='Word_filename')

print(df.head)
list_of_words = []
list_of_index = []

for row in df.itertuples():
    list_of_words.append(row.WORD)
    list_of_index.append(row._2)
    
    
folder = "uaspeech_experiments_M5/"
paths = ['M05_0.75x/', 'M05_1.25x/', 'M05_1.5x/', 'M05_1.75x/', 'M05_2.0x/', 'M05_1.0x/']
file_name = 'M05_wav_list.txt'
save_location = "predictions_m05/"

for path in paths:
    loc = folder + path + file_name
    
    with open(loc) as f:
        lines = f.readlines()
    
    list_of_transcriptions = []
    
    for name in lines:
        for i in range(len(list_of_words)):
            string1 = "_" + list_of_index[i] + "_"
            if string1 in name:
                list_of_transcriptions.append(list_of_words[i])
    
    save_loc = save_location + path[:-1] + "_transcription.txt"
    with open(save_loc, 'w') as output:
        for transcription in list_of_transcriptions:
            output.write(transcription)
            output.write("\n")

<bound method NDFrame.head of        WORD FILE NAME
0     Three        D3
1      Nine        D9
2      Zero        D0
3       Six        D6
4     Seven        D7
..      ...       ...
450    fire   B3_UW96
451   watch   B3_UW97
452   ahead   B3_UW98
453    away   B3_UW99
454  crayon  B3_UW100

[455 rows x 2 columns]>


## WER for speaker M05

In [65]:
folder = "predictions_m05/"
paths = ['M05_0.75x', 'M05_1.0x', 'M05_1.25x', 'M05_1.5x', 'M05_1.75x', 'M05_2.0x']
adds = ["_predictions_original.txt", "_predictions_transformed.txt", "_transcription.txt"]

lowest = 1000
path_lowest = "path"
for path in paths:
    transcriptions = open_folder_and_return_lines(folder + paths[0] + adds[2])
    original = open_folder_and_return_lines(folder + path + adds[0])
    converted = open_folder_and_return_lines(folder + path + adds[1])
    
    print(path)
    print("original")
    print(calculate_wer(original, transcriptions))
    print("\n")
    
    print(path)
    print("converted")
    print(calculate_wer(converted, transcriptions))
    print("\n")
    
    if calculate_wer(original, transcriptions) < lowest:
        path_lowest = path + "original"
        lowest =  calculate_wer(original, transcriptions)
    if calculate_wer(converted, transcriptions) < lowest:
        path_lowest = path + "converted"
        lowest = calculate_wer(converted, transcriptions)

print("lowest value")
print(path_lowest, lowest)


M05_0.75x
original
249.92


M05_0.75x
converted
317.09


M05_1.0x
original
241.34


M05_1.0x
converted
288.52


M05_1.25x
original
248.12


M05_1.25x
converted
261.68


M05_1.5x
original
238.32


M05_1.5x
converted
236.69


M05_1.75x
original
233.56


M05_1.75x
converted
216.08


M05_2.0x
original
219.83


M05_2.0x
converted
199.83


lowest value
M05_2.0xconverted 199.83


## CER for speaker M05

In [66]:
folder = "predictions_m05/"
paths = ['M05_0.75x', 'M05_1.0x', 'M05_1.25x', 'M05_1.5x', 'M05_1.75x', 'M05_2.0x']
adds = ["_predictions_original.txt", "_predictions_transformed.txt", "_transcription.txt"]

lowest = 1000
path_lowest = "path"
for path in paths:
    transcriptions = open_folder_and_return_lines(folder + paths[0] + adds[2])
    original = open_folder_and_return_lines(folder + path + adds[0])
    converted = open_folder_and_return_lines(folder + path + adds[1])
    
    print(path)
    print("original")
    print(calculate_wer_character_level(original, transcriptions))
    print("\n")
    
    print(path)
    print("converted")
    print(calculate_wer_character_level(converted, transcriptions))
    print("\n")
    
    if calculate_wer_character_level(original, transcriptions) < lowest:
        path_lowest = path + "original"
        lowest =  calculate_wer_character_level(original, transcriptions)
    if calculate_wer_character_level(converted, transcriptions) < lowest:
        path_lowest = path + "converted"
        lowest = calculate_wer_character_level(converted, transcriptions)

print("lowest value")
print(path_lowest, lowest)

M05_0.75x
original
170.81


M05_0.75x
converted
223.73


M05_1.0x
original
158.42


M05_1.0x
converted
196.1


M05_1.25x
original
160.06


M05_1.25x
converted
175.48


M05_1.5x
original
151.42


M05_1.5x
converted
156.48


M05_1.75x
original
148.95


M05_1.75x
converted
143.87


M05_2.0x
original
138.7


M05_2.0x
converted
133.53


lowest value
M05_2.0xconverted 133.53
